In [1]:
from librarymaster import *

2024-11-22 18:01:03.829532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732278663.846761  119688 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732278663.852115  119688 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 18:01:03.868960: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df=pd.read_csv('../materials/paired_nl_bash.csv',delimiter=',')
# df.head()


In [3]:
input_texts = df['natural_language']
# input_texts.head()

In [4]:
output_texts = df['bash_command']
# output_texts.head()

In [5]:
SPECIAL_TOKENS = ['<start>', '<end>', '<unk>', '<pad>']
#Tokenise and pad sequences
# input_tokeniser = Tokenizer()
# output_tokeniser = Tokenizer()
input_tokeniser = Tokenizer(oov_token='<unk>')  # Adding out-of-vocabulary token
output_tokeniser = Tokenizer(oov_token='<unk>')

In [6]:
input_tokeniser.fit_on_texts(input_texts)
# output_tokeniser.fit_on_texts(output_texts)
output_texts_with_tokens = ['<start> ' + text + ' <end>' for text in output_texts]
output_tokeniser.fit_on_texts(output_texts_with_tokens)

In [22]:
if '<start>' not in output_tokeniser.word_index:
    print("Warning: Special tokens not in vocabulary. Adding them...")
    # Add special tokens to vocabulary if they're missing
    current_vocab_size = len(output_tokeniser.word_index)
    for i, token in enumerate(SPECIAL_TOKENS, start=1):
        if token not in output_tokeniser.word_index:
            output_tokeniser.word_index[token] = current_vocab_size + i
            output_tokeniser.index_word[current_vocab_size + i] = token

NameError: name 'SPECIAL_TOKENS' is not defined

In [8]:
# # input_vocabsize, output_vocabsize
input_vocabsize = len(input_tokeniser.word_index) + 1
output_vocabsize = len(output_tokeniser.word_index) + 1

In [7]:
#Sequence processing
input_sequences = pad_sequences(input_tokeniser.texts_to_sequences(input_texts), 
                              padding='post')
output_sequences = pad_sequences(output_tokeniser.texts_to_sequences(output_texts_with_tokens),
                               padding='post')

In [9]:
# Model definition
embedding_dim=128
units=256

In [10]:
# # Encoder
# encoder_inputs = tf.keras.Input(shape=(None,))
# encoder_embedding = Embedding(input_vocabsize, embedding_dim)(encoder_inputs)
# encoder_lstm = LSTM(units, return_state=True)
# encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

In [11]:
# #Decoder with attention mechanism
# decoder_inputs = tf.keras.Input(shape=(None,))
# decoder_embedding = Embedding(output_vocabsize, embedding_dim)(decoder_inputs)
# decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_embedding, 
#                                    initial_state=[state_h, state_c])
# decoder_dense = Dense(output_vocabsize, activation='softmax')
# output = decoder_dense(decoder_outputs)

In [12]:
# # Training Model
# model = Model([encoder_inputs, decoder_inputs], output)
# model.compile(optimizer='adam', 
#              loss='sparse_categorical_crossentropy',
#              metrics=['accuracy'])

In [13]:
def create_training_model():
    # Encoder
    encoder_inputs = tf.keras.Input(shape=(None,))
    encoder_embedding = Embedding(input_vocabsize, embedding_dim, name='embedding')(encoder_inputs)
    encoder_lstm = LSTM(units, return_state=True, name='lstm')
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

    # Decoder
    decoder_inputs = tf.keras.Input(shape=(None,))
    decoder_embedding = Embedding(output_vocabsize, embedding_dim, name='embedding_1')(decoder_inputs)
    decoder_lstm = LSTM(units, return_sequences=True, return_state=True, name='lstm_1')
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
    decoder_dense = Dense(output_vocabsize, activation='softmax', name='dense')
    output = decoder_dense(decoder_outputs)

    # Create and compile model
    model = Model([encoder_inputs, decoder_inputs], output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [14]:
#Inference Setup
def create_inference_models(trained_model):
    # Get the layers from trained model
    encoder_inputs = trained_model.input[0]
    encoder_embedding = trained_model.get_layer('embedding')
    encoder_lstm = trained_model.get_layer('lstm')
    
    # Recreate encoder model
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding(encoder_inputs))
    encoder_model = Model(encoder_inputs, [state_h, state_c])
    
    # Decoder setup
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = trained_model.get_layer('embedding_1')
    decoder_lstm = trained_model.get_layer('lstm_1')
    decoder_dense = trained_model.get_layer('dense')
    
    decoder_state_input_h = Input(shape=(units,))
    decoder_state_input_c = Input(shape=(units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs = decoder_embedding(decoder_inputs)
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_outputs, initial_state=decoder_states_inputs
    )
    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs, state_h, state_c]
    )
    
    return encoder_model, decoder_model


In [15]:
def generate_command(input_query, max_length=50):
    # Input preprocessing
    input_seq = input_tokeniser.texts_to_sequences([input_query])
    input_seq = pad_sequences(input_seq, maxlen=input_sequences.shape[1], padding='post')
    
    # Get initial states from encoder
    states_value = encoder_model.predict(input_seq, verbose=0)
    
    # Initialize target sequence with start token
    target_seq = np.zeros((1, 1))
    start_token_idx = output_tokeniser.word_index.get('<start>', 1)  # Default to 1 if not found
    target_seq[0, 0] = start_token_idx
    
    decoded_sentence = []
    
    while True:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = output_tokeniser.index_word.get(sampled_token_index, '<unk>')
        
        if sampled_word == '<end>' or len(decoded_sentence) > max_length:
            break
            
        if sampled_word not in SPECIAL_TOKENS:  # Skip all special tokens
            decoded_sentence.append(sampled_word)
        
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    
    return ' '.join(decoded_sentence)

#### Enable this by uncommenting only when you need it to train

In [16]:
# Create and train the model
model = create_training_model()
model.fit(
    [input_sequences, output_sequences[:, :-1]],
    output_sequences[:, 1:],
    epochs=20,
    batch_size=64,
    validation_split=0.2
)

Epoch 1/20


2024-11-22 18:01:09.220768: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


158/158 ━━━━━━━━━━━━━━━━━━━━ 101s 622ms/step - accuracy: 0.8600 - loss: 2.4384 - val_accuracy: 0.9029 - val_loss: 0.6260
Epoch 2/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 100s 630ms/step - accuracy: 0.8948 - loss: 0.6691 - val_accuracy: 0.9063 - val_loss: 0.6003
Epoch 3/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 102s 647ms/step - accuracy: 0.8986 - loss: 0.6392 - val_accuracy: 0.9091 - val_loss: 0.5774
Epoch 4/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 106s 671ms/step - accuracy: 0.9000 - loss: 0.6110 - val_accuracy: 0.9098 - val_loss: 0.5581
Epoch 5/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 105s 668ms/step - accuracy: 0.9015 - loss: 0.5878 - val_accuracy: 0.9114 - val_loss: 0.5418
Epoch 6/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 106s 672ms/step - accuracy: 0.9027 - loss: 0.5710 - val_accuracy: 0.9119 - val_loss: 0.5307
Epoch 7/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 106s 669ms/step - accuracy: 0.9053 - loss: 0.5450 - val_accuracy: 0.9145 - val_loss: 0.5176
Epoch 8/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 105s 665ms/step - accuracy: 0.9081 - loss: 0.52

In [20]:
# Create inference models
encoder_model, decoder_model = create_inference_models(model)

In [21]:
# Test the model
input_query = "(BSD specific) Display process information twice, waiting one second between each, filtering out the header line."
generated_command = generate_command(input_query)
print(f"Generated Command: {generated_command}")

KeyError: '<start>'

In [ ]:
# # # Train the model
# model.fit(
#     [input_sequences, output_sequences[:, :-1]],
#     output_sequences[:, 1:],
#     batch_size=64,
#     epochs=20,
#     validation_split=0.2
# )

In [17]:
# save_model_and_tokenizers(model, input_tokeniser, output_tokeniser, 'command_generator')


In [20]:
# def create_training_model():
#     # Encoder
#     encoder_inputs = tf.keras.Input(shape=(None,))
#     encoder_embedding = Embedding(input_vocabsize, embedding_dim, name='embedding')(encoder_inputs)
#     encoder_lstm = LSTM(units, return_state=True, name='lstm')
#     encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

#     # Decoder
#     decoder_inputs = tf.keras.Input(shape=(None,))
#     decoder_embedding = Embedding(output_vocabsize, embedding_dim, name='embedding_1')(decoder_inputs)
#     decoder_lstm = LSTM(units, return_sequences=True, return_state=True, name='lstm_1')
#     decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
#     decoder_dense = Dense(output_vocabsize, activation='softmax', name='dense')
#     output = decoder_dense(decoder_outputs)

#     # Create and compile model
#     model = Model([encoder_inputs, decoder_inputs], output)
#     model.compile(optimizer='adam', 
#                  loss='sparse_categorical_crossentropy',
#                  metrics=['accuracy'])
#     return model

In [ ]:
# # Create inference models
# encoder_model, decoder_model = create_inference_models()

In [ ]:
# # Generate commands
# input_query = "(BSD specific) Display process information twice, waiting one second between each, filtering out the header line."
# generated_command = generate_command(input_query)
# print(f"Generated Command: {generated_command}")

In [14]:
# def decode_sequence(input_text):
#     #Tokenize input
#     input_seq = pad_sequences(input_tokeniser.texts_to_sequences([input_texts]),
#                               maxlen=input_sequences.shape[1])
#     states_value=encoder_model.predict(input_seq)

#     #Start token for ecoder
#     target_seq=np.zeros((1,1))
#     target_seq[0,0]=output_tokeniser.word_index['<start>']

#     #Loop to generate command
#     generated_command=''
#     for _ in range(max_length):
#         output_tokens,h,c=decoder_model.predict([target_seq]+states_value)
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         sampled_word = output_tokeniser.index_word[sampled_token_index]

#         if sampled_word == '<end>':
#             break

#         generated_command += ' ' + sampled_word

#         target_seq = np.zeros((1, 1))
#         target_seq[0, 0] = sampled_token_index

#         states_value = [h, c]

#     return generated_command

In [15]:
# #saving the model
# model.save('genCommand.keras')

In [16]:
# # Saving the input tokenizer
# with open('input_tokeniser.pkl', 'wb') as handle:
#     pickle.dump(input_tokeniser, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # Saving the output tokenizer
# with open('output_tokeniser.pkl', 'wb') as handle:
#     pickle.dump(output_tokeniser, handle, protocol=pickle.HIGHEST_PROTOCOL)


## Inference

In [ ]:
# model = load_model('genCommand.keras')

In [18]:
# # Encoder setup
# encoder_inputs = model.input[0]  # Input for the encoder
# encoder_outputs, state_h, state_c = model.layers[4].output  # Outputs from the encoder LSTM
# encoder_model = Model(encoder_inputs, [state_h, state_c])  # Define the encoder model

In [19]:
# # Define decoder input sequence
# decoder_inputs = Input(shape=(None,))  # Placeholder for decoder input sequences
# decoder_embedding = model.layers[3](decoder_inputs)  # Embedding layer for decoder

# # Retrieve LSTM units
# units = model.layers[5].units

# # Placeholders for decoder LSTM states
# decoder_state_input_h = Input(shape=(units,))
# decoder_state_input_c = Input(shape=(units,))
# decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# # Recreate the decoder LSTM layer for inference
# decoder_lstm_inference = LSTM(
#     units,
#     return_sequences=True,
#     return_state=True,
#     name="decoder_lstm_inference",
# )
# decoder_outputs, state_h, state_c = decoder_lstm_inference(
#     decoder_embedding, initial_state=decoder_states_inputs
# )

# # Dense layer for output
# decoder_dense = model.layers[6]
# decoder_outputs = decoder_dense(decoder_outputs)

# # Define the decoder model
# decoder_model = Model(
#     [decoder_inputs] + decoder_states_inputs,  # Inputs for decoder sequence and states
#     [decoder_outputs, state_h, state_c],  # Outputs: predictions and new states
# )


In [20]:
# output_tokeniser.word_index['<start>'] = len(output_tokeniser.word_index) + 1
# output_tokeniser.word_index['<end>'] = len(output_tokeniser.word_index) + 1
# output_tokeniser.index_word[output_tokeniser.word_index['<start>']] = '<start>'
# output_tokeniser.index_word[output_tokeniser.word_index['<end>']] = '<end>'

In [ ]:
# print('<start>' in output_tokeniser.word_index)  # Should return True
# print('<end>' in output_tokeniser.word_index)    # Should return True


In [22]:

# def generate_command(input_text, max_length=50):
#     # Tokenize and pad the input
#     input_seq = pad_sequences(
#         input_tokeniser.texts_to_sequences([input_text]),
#         maxlen=input_sequences.shape[1],  # Ensure padding matches the trained input size
#         padding='post'
#     )

#     # Encode the input
#     states_value = encoder_model.predict(input_seq)

#     # Start the decoding process
#     # Start the decoding process
#     start_token_id = output_tokeniser.word_index['<start>']
#     target_seq = pad_sequences([[start_token_id]], maxlen=1)
#     decoded_sentence = ''

    
#     for _ in range(max_length):
#         # Predict the next token
#         output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
#         # Get the most likely token
#         sampled_token_index = output_tokens.argmax(axis=-1)[0, -1]
#         sampled_word = output_tokeniser.index_word[sampled_token_index]
        
#         # Break if the end token is reached
#         if sampled_word == '<end>':
#             break

#         decoded_sentence += ' ' + sampled_word
        
#         # Update the target sequence
#         target_seq = pad_sequences([[sampled_token_index]], maxlen=1, padding='post')

#         # Update states
#         states_value = [h, c]
    
#     return decoded_sentence.strip()


In [ ]:
# print(output_tokeniser.word_index)


### Start

In [24]:
# # If the embedding layer has a name
# embedding_layer = model.get_layer('embedding')

In [ ]:
# print(f"Embedding vocabulary size: {embedding_layer.input_dim}")

In [ ]:
# # Check the embedding layer's vocabulary size
# print(f"Embedding vocabulary size: {embedding_layer.input_dim}")

# # Example: Tokenize the input query
# input_query = "(BSD specific) Display process information twice, waiting one second between each, filtering out the header line."
# tokenized_input = input_tokeniser.texts_to_sequences([input_query])  # Ensure you use the trained tokenizer
# print(f"Max token index in input: {max(tokenized_input[0])}")

# # Validate input tokens
# if max(tokenized_input[0]) >= embedding_layer.input_dim:
#     print("Error: Input token exceeds embedding layer vocabulary size.")

In [27]:
# encoder_max_seq_length = encoder_model.input_shape[1]
# # encoder_max_seq_length,  target_tokeniser.word_index['<start>']

In [32]:
# def generate_command(input_query, max_length=50):
#     # Add input validation
#     tokenized_input = input_tokeniser.texts_to_sequences([input_query])
#     if not tokenized_input[0]:  # Check if tokenization produced empty sequence
#         return "Error: Could not tokenize input"
    
#     # Ensure all token indices are within bounds
#     if max(tokenized_input[0]) >= input_vocabsize:
#         return "Error: Input contains unknown tokens"
        
#     tokenized_input = pad_sequences(tokenized_input, 
#                                   maxlen=encoder_max_seq_length, 
#                                   padding='post')
    
#     # Get encoder states
#     states_value = encoder_model.predict(tokenized_input, verbose=0)
    
#     # Initialize with start token
#     target_seq = np.zeros((1, 1))
#     start_token_idx = output_tokeniser.word_index.get('<start>', 1)  # Default to 1 if not found
#     target_seq[0, 0] = start_token_idx
    
#     decoded_sentence = []
    
#     for _ in range(max_length):
#         output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
        
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         sampled_word = output_tokeniser.index_word.get(sampled_token_index, '<unk>')
        
#         if sampled_word == '<end>':
#             break
            
#         if sampled_word != '<unk>':
#             decoded_sentence.append(sampled_word)
            
#         target_seq = np.zeros((1, 1))
#         target_seq[0, 0] = sampled_token_index
#         states_value = [h, c]
    
#     return ' '.join(decoded_sentence)

### End

In [ ]:
# input_query = "(BSD specific) Display process information twice, waiting one second between each, filtering out the header line."
# generated_command = generate_command(input_query)
# print(f"Generated Command: {generated_command}")